# The Attribution CLI

We could also run the steps from the `AttributionPackageExample.ipynb` using the CLI (Command Line Interface) which will run the requested functions from the package directly in the terminal and save it to a file. You could run a `climattr --help` to get the list of functions:

```
usage: climattr [-h]
                {filter-area,filter-time,attr-metrics,attr-plot,qq-plot,validation-plot,exploratory-plot,xclim-indice,scale}
                ...

Run Extreme Event Attribution

positional arguments:
  {filter-area,filter-time,attr-metrics,attr-plot,qq-plot,validation-plot,exploratory-plot,xclim-indice,scale}
    filter-area         Function used to filter the dataset based on a spatial filter
    filter-time         Function used to filter the dataset based on initial and final time
    attr-metrics        Function used calculate the attribution metrics (PR, RP, FAR)
    attr-plot           Function used to plot attribution histogram and RP
    qq-plot             Function used for QQ-Plot against theoretical quantiles
    validation-plot     Function used for validation plot of OBS and ALL
    exploratory-plot    Function used to explore the plots for OBS
    xclim-indice        Function used to wrap xclim library and calculate the indice
    scale               Function used to scale data by a given climatology (add or divide)

options:
  -h, --help            show this help message and exit
```

Also if you ask for the helper for a specific function:

```
usage: climattr attr-metrics [-h] -a ALL -n NAT -o OFILE -d {multi-file,single-file} -v VARIABLE [-f FIT_FUNCTION]
                             [-t THRESH] [--direction {descending,ascending}]

options:
  -h, --help            show this help message and exit
  -a ALL, --all ALL     Path to the 'all' dataset files
  -n NAT, --nat NAT     Path to the 'nat' dataset files
  -o OFILE, --ofile OFILE
                        Path to the output dataset
  -d {multi-file,single-file}, --data-source {multi-file,single-file}
                        Specify the data source type: 'multi-file' or 'single-file'
  -v VARIABLE, --variable VARIABLE
                        Variable name to use for attribution metrics (default: 'tas')
  -f FIT_FUNCTION, --fit_function FIT_FUNCTION
                        Scipy fit function to use for attribution metrics (default: 'norm')
  -t THRESH, --thresh THRESH
                        Threshold value for the attribution metrics (default: 301)
  --direction {descending,ascending}
                        Direction for the bootstrap ordering that will be used to calculate RP
```

Replicating what we have on the `AttributionPackageExample.ipynb` we have:

In [ ]:
# Preprocessing OBS
!climattr filter-time --ifile '/media/rafael/Rafael/cmip6/jasmin/era5/amazon/era5_dataset.1981.2023.daily.t2m.nc' -d single-file -v t2m --itime '1981-01-01' --etime '2014-02-28' --months 12 1 2 --ofile obs1.nc
!climattr filter-area --ifile obs1.nc -d single-file -v t2m --mask /home/rafael/Downloads/amazonas.shp --reduce mean --ofile obs2.nc
!climattr xclim-indice --ifile obs2.nc -d single-file --xclim-function tx_max -k tasmax="t2m" freq="YE-NOV" --ofile obs3.nc

# Plot historical data exploratory plot
!climattr exploratory-plot --i obs3.nc -d single-file -v tx_max -f genextreme -y 2003 --ofile exploratory.png

In [ ]:
# Preprocessing ALL
!climattr filter-time --ifile '/media/rafael/Rafael/cmip6/jasmin/amazon/time/tasmax_day_IPSL-CM6A-LR_historical_*.nc' -d multi-file -v tasmax  --itime '1981-01-01' --etime '2014-02-28' --months 12 1 2 --ofile all1.nc
!climattr filter-area --ifile all1.nc -d single-file -v tasmax --mask /home/rafael/Downloads/amazonas.shp --reduce mean --ofile all2.nc
!climattr xclim-indice --ifile all2.nc -d single-file --xclim-function tx_max -k tasmax="tasmax" freq="YE-NOV" --ofile all3.nc

# QQ-Plot for validation
!climattr qq-plot --all all3.nc --obs obs3.nc -d single-file -v tx_max -f genextreme --ofile qq_plot.png 

# Histogram and 2 sample QQ-Plot for validation
!climattr validation-plot --all all3.nc --obs obs3.nc -d single-file -v tx_max -f genextreme --ofile validation_plot.png 

In [ ]:
# Scale data
!climattr scale --ifile obs3.nc --clim obs3.nc -d single-file -v tx_max --idate '1981-01-01' --edate '2010-12-31' --method add --ofile obs4.nc
!climattr scale --ifile all3.nc --clim all3.nc -d single-file -v tx_max --idate '1981-01-01' --edate '2010-12-31' --method add --ofile all4.nc

# plot again the validation plot
!climattr validation-plot --all all4.nc --obs obs4.nc -d single-file -v tx_max -f genextreme --ofile validation_plot_scaled.png 

In [ ]:
# Preprocessing NAT
!climattr filter-time --ifile '/media/rafael/Rafael/cmip6/jasmin/amazon/time/tasmax_day_IPSL-CM6A-LR_hist-nat_*.nc' -d multi-file -v tasmax  --itime '1981-01-01' --etime '2014-02-28' --months 12 1 2 --ofile nat1.nc
!climattr filter-area --ifile nat1.nc -d single-file -v tasmax --mask /home/rafael/Downloads/amazonas.shp --reduce mean --ofile nat2.nc
!climattr xclim-indice --ifile nat2.nc -d single-file --xclim-function tx_max -k tasmax="tasmax" freq="YE-NOV" --ofile nat3.nc

# Scale NAT
!climattr scale --ifile nat3.nc --clim all3.nc -d single-file -v tx_max --idate '1981-01-01' --edate '2010-12-31' --method add --ofile nat4.nc

In [ ]:
# Attribution plot
!climattr attr-plot --all all4.nc --nat nat4.nc -d single-file -v tx_max -f genextreme -t 1.545 --direction descending --ofile attribution_plot.png

# Attribution metrics
!climattr attr-metrics --all all4.nc --nat nat4.nc -d single-file -v tx_max -f genextreme -t 1.545 --direction descending --ofile attribution_metrics.csv